<h1>Movie Description Text Pre-Processing</h1>

In [1]:
import pandas as pd
import requests
import json
import unidecode
# import goslate
# import sys
# import os
# # project_root = "D:\\Leaning_NLP"
# # sys.path.append(os.path.abspath(os.path.join(os.path.dirname(project_root), 'polyglot')))
# import polyglot
# from polyglot.text import Text
# from transliterate import translit, get_available_language_codes

<h2>Collecting Data</h2>

In [2]:
URL = "https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page="
movies = []
for i in range(100): 
    print(f"\rCollecting movies from page : {i+1}", end = '\r')
    lst = json.loads(requests.get(URL + f"{i+1}").text)['results']
    for movie in lst: 
        movies.append(movie)
print("Collection Completed!!        ", end = "\n")    

Collection Completed!!        100


In [3]:
# movies[0]
GENRE_URL = "https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US"
lst = json.loads(requests.get(GENRE_URL).text)['genres']
mp = {}
for item in lst:
    mp[item['id']] = item['name']

In [4]:
lst = []
for movie in movies:
    txt = {}
    txt['original_language'] = movie['original_language']
    txt['original_title'] = movie['original_title']
    # if txt['original_language'] != 'en':
    #     txt['original_title'] = unidecode.unidecode(movie['original_title']).lower()
    txt['overview'] = movie['overview']
    txt['genre'] = []
    for id in movie['genre_ids']:
        txt['genre'].append(mp[id])
    lst.append(txt)
    
df = pd.DataFrame(lst)
df.head(10)

,original_language,original_title,overview,genre
0,en,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[Drama, Crime]"
1,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[Drama, Crime]"
2,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[Drama, Crime]"
3,en,Schindler's List,The true story of how businessman Oskar Schind...,"[Drama, History, War]"
4,en,12 Angry Men,The defense and the prosecution have rested an...,[Drama]
5,hi,दिलवाले दुल्हनिया ले जायेंगे,"Raj is a rich, carefree, happy-go-lucky second...","[Comedy, Drama, Romance]"
6,ja,千と千尋の神隠し,"A young girl, Chihiro, becomes trapped in a st...","[Animation, Family, Fantasy]"
7,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[Drama, Action, Crime, Thriller]"
8,ko,기생충,"All unemployed, Ki-taek's family takes peculia...","[Comedy, Thriller, Drama]"
9,en,The Green Mile,A supernatural tale set on death row in a Sout...,"[Fantasy, Drama, Crime]"


In [5]:
# df['original_language'].value_counts()

In [6]:
for r in range(6):
    print(df['original_title'][r])

The Shawshank Redemption
The Godfather
The Godfather Part II
Schindler's List
12 Angry Men
दिलवाले दुल्हनिया ले जायेंगे


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = Options()
chrome_options.add_experimental_option("detach",True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
driver.maximize_window()


from bs4 import BeautifulSoup
import requests

URL = "https://www.themoviedb.org/search?query="

def convert_english(qury):
    try:
        html_text = requests.get(URL + qury).text
        soup = BeautifulSoup(html_text, 'lxml')
        res_box = soup.find('div', class_ = 'results flex')
        res = res_box.find('div', class_ = 'wrapper')
        res = res.find('a')
        # print(res['href'])
        href = res['href']
        forv_url = "https://www.themoviedb.org/"
        href = forv_url + href
        html_text = requests.get(href).text
        soup = BeautifulSoup(html_text, 'lxml')
        res_lang = soup.find('html')['lang']
        if(res_lang != 'en'):
            driver.get(href)
            # button = driver.find_element('xpath', '/html/body/div[1]/header/div[1]/div/div[2]/ul/li[2]')
            # button.click()
            # button = driver.find_element('xpath', "//*[contains(text(), 'English (en-US)')]")
            html_text = driver.page_source
            soup = BeautifulSoup(html_text, 'lxml')
        res = soup.find('section', class_ = 'header poster')
        res = res.find('h2')
        return res.text
    except:
        print("Issue With", qury)
        return qury
        
# print(convert_english('기생충'))

# df['original_title'] = df.apply(lambda x : convert_english(x['original_title'], x['original_language']), axis = 1)

# for i in range(df.shape[0])
for i in range(100):
    if df['original_language'][i] in ['en', 'fr', 'it', 'es', 'de']:
        continue
    df['original_title'][i] = convert_english(df['original_title'][i])

df.head(10)

driver.quit()

C:\Users\HP\AppData\Local\Temp\ipykernel_1940\1086801729.py:58: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['original_title'][i] = convert_english(df['original_title'][i])


In [8]:
for i in range(df.shape[0]):
    df.iloc[i, 1] = df.iloc[i, 1].replace('\n', " ")
df.head(10)

,original_language,original_title,overview,genre
0,en,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[Drama, Crime]"
1,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[Drama, Crime]"
2,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[Drama, Crime]"
3,en,Schindler's List,The true story of how businessman Oskar Schind...,"[Drama, History, War]"
4,en,12 Angry Men,The defense and the prosecution have rested an...,[Drama]
5,hi,Dilwale Dulhania Le Jayenge (1995),"Raj is a rich, carefree, happy-go-lucky second...","[Comedy, Drama, Romance]"
6,ja,Spirited Away (2001),"A young girl, Chihiro, becomes trapped in a st...","[Animation, Family, Fantasy]"
7,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,"[Drama, Action, Crime, Thriller]"
8,ko,Parasite (2019),"All unemployed, Ki-taek's family takes peculia...","[Comedy, Thriller, Drama]"
9,en,The Green Mile,A supernatural tale set on death row in a Sout...,"[Fantasy, Drama, Crime]"


<!-- ![alt text]("Screenshot 2024-06-09 203944.png") -->
<h2>PreProcessing Text</h2>
<h4>We will do preprocessing of data on the overview tag</h4>
<h4>Method Used:</h4>
<img src = "Screenshot 2024-06-09 203944.png">

<h3>Lowercasing</h3>

In [9]:
df['overview'] = df['overview'].apply(lambda x : x.lower())
df.sample(10)

,original_language,original_title,overview,genre
1011,en,Out of the Past,jeff bailey seems to be a mundane gas station ...,"[Crime, Romance, Thriller]"
352,en,Where Hands Touch,"germany, 1944. leyna, the 15-year old daughter...","[War, Drama, Romance]"
1779,en,Fantasia,walt disney's timeless masterpiece is an extra...,"[Animation, Family, Fantasy]"
1806,en,Uncle Frank,"in 1973, when frank bledsoe and his 18-year-ol...",[Drama]
1181,en,Fury,"in the last months of world war ii, as the all...","[War, Drama, Action]"
1272,ja,レッドライン,the most dangerous and exciting car race in th...,"[Animation, Action, Science Fiction]"
565,en,Snatch,"unscrupulous boxing promoters, violent bookmak...","[Crime, Comedy]"
1656,en,Mad Max 2,max rockatansky returns as the heroic loner wh...,"[Adventure, Action, Thriller, Science Fiction]"
997,fr,Au hasard Balthazar,the story of a donkey balthazar as he is passe...,[Drama]
1137,en,Me and Earl and the Dying Girl,greg is coasting through senior year of high s...,"[Drama, Comedy]"


<h3>Remove HTML tags</h3>

In [10]:
import re
def tags_remover(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

df['overview'] = df['overview'].apply(lambda x : tags_remover(x))
df.sample(10)

,original_language,original_title,overview,genre
1718,en,The Lady from Shanghai,a romantic drifter gets caught between a corru...,"[Mystery, Crime, Thriller]"
124,ja,劇場版 呪術廻戦 0,yuta okkotsu is a nervous high school student ...,"[Animation, Action, Fantasy]"
863,pt,Minha Mãe é uma Peça 2: O Filme,"dona hermínia is back, but now rich and famous...",[Comedy]
627,de,Werk ohne Autor,german artist kurt barnert has escaped east ge...,"[Drama, Romance, History]"
1983,ja,GANTZ:O,after being brutally murdered in a subway stat...,"[Science Fiction, Action, Animation]"
905,en,The Lost Weekend,"don birnam, a long-time alcoholic, has been so...",[Drama]
410,en,Captain Fantastic,a father living in the forests of the pacific ...,"[Adventure, Drama]"
1619,it,Caro diario,nanni moretti recalls in his diary three slice...,"[Comedy, Drama]"
756,en,Inherit the Wind,schoolteacher bertram cates is arrested for te...,[Drama]
859,pl,Boże Ciało,a pious 20-year-old juvenile delinquent is sen...,[Drama]


<h3>Remove URLs </h3>

In [11]:
def remove_urls(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

df['overview'] = df['overview'].apply(remove_urls)
df.sample(10)

,original_language,original_title,overview,genre
376,en,Flamin' Hot,"the inspiring true story of richard montañez, ...","[Comedy, Drama]"
128,es,Voces inocentes,"a young boy, in an effort to have a normal chi...","[Drama, War]"
948,en,Mickey's Christmas Carol,ebenezer scrooge is far too greedy to understa...,"[Family, Animation]"
612,en,Everything Everywhere All at Once,an aging chinese immigrant is swept up in an i...,"[Action, Adventure, Science Fiction]"
620,en,Dune,"paul atreides, a brilliant and gifted young ma...","[Science Fiction, Adventure]"
657,ko,범죄도시,"in chinatown, law and order is turned upside d...","[Action, Crime, Thriller]"
1198,en,"One, Two, Three",c.r. macnamara is a managing director for coca...,[Comedy]
833,en,What's Eating Gilbert Grape,gilbert grape is a small-town young man with a...,"[Romance, Drama]"
1548,en,Doctor Strange,"after his career is destroyed, a brilliant but...","[Action, Adventure, Fantasy]"
1191,cn,警察故事,officer chan ka kui manages to put a major hon...,"[Action, Crime, Thriller]"


<h3>Remove Punctuation</h3>

In [12]:
# punctuation_marks = '"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~'
import string

punctuation_marks = string.punctuation

# We excluded '!' & '@' from punctuations as it could give useful information. 
punctuation_marks = punctuation_marks[1:20] + punctuation_marks[21:]

def remove_punctuations(text):
    return text.translate(str.maketrans(' ', ' ', punctuation_marks))

df['overview'] = df['overview'].apply(remove_punctuations)
df.head(10)

,original_language,original_title,overview,genre
0,en,The Shawshank Redemption,imprisoned in the 1940s for the double murder ...,"[Drama, Crime]"
1,en,The Godfather,spanning the years 1945 to 1955 a chronicle of...,"[Drama, Crime]"
2,en,The Godfather Part II,in the continuing saga of the corleone crime f...,"[Drama, Crime]"
3,en,Schindler's List,the true story of how businessman oskar schind...,"[Drama, History, War]"
4,en,12 Angry Men,the defense and the prosecution have rested an...,[Drama]
5,hi,Dilwale Dulhania Le Jayenge (1995),raj is a rich carefree happygolucky second gen...,"[Comedy, Drama, Romance]"
6,ja,Spirited Away (2001),a young girl chihiro becomes trapped in a stra...,"[Animation, Family, Fantasy]"
7,en,The Dark Knight,batman raises the stakes in his war on crime w...,"[Drama, Action, Crime, Thriller]"
8,ko,Parasite (2019),all unemployed kitaeks family takes peculiar i...,"[Comedy, Thriller, Drama]"
9,en,The Green Mile,a supernatural tale set on death row in a sout...,"[Fantasy, Drama, Crime]"


<h3>Chat Word Treatment</h3>

In [13]:
import csv

# Git hub link for the same can be found here : https://github.com/rishabhverma17/sms_slang_translator
def chat_word_converter(user_string):
    user_string = user_string.split(" ")
    j = 0
    for _str in user_string:
        # File path which consists of Abbreviations.
        fileName = "slang.txt"
        # File Access mode [Read Mode]
        accessMode = "r"
        with open(fileName, accessMode) as myCSVfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            # Removing Special Characters.
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if _str.upper() == row[0]:
                    # If match found replace it with its Abbreviation in text file.
                    user_string[j] = row[1]
            myCSVfile.close()
        j = j + 1
    # Replacing commas with spaces for final output.
    return ' '.join(user_string).lower()

# print(chat_word_converter("Hi bro ltns"))

df['overview'] = df['overview'].apply(chat_word_converter)
df.sample(5)

,original_language,original_title,overview,genre
1170,en,The Man Who Wasn't There,a tale of murder crime and punishment set in t...,"[Crime, Drama]"
152,en,Singin' in the Rain,in 1927 hollywood a silent film production com...,"[Comedy, Romance]"
1606,en,The Omen,immediately after their miscarriage the us dip...,"[Horror, Thriller]"
1357,en,Bullet Train,unlucky assassin ladybug is determined to do h...,"[Action, Comedy, Thriller]"
1914,en,Batman: Assault on Arkham,batman works desperately to find a bomb plante...,"[Thriller, Animation, Action, Crime]"


<h3>Spelling Correction Algorithms</h3>

In [14]:
from textblob import TextBlob
from spellchecker import SpellChecker
spell = SpellChecker()

def spelling_correction(text):
    new_text = TextBlob(text)
    return new_text.correct()

print(spelling_correction("gunda"))

guns


<h4>I use here pyspellchecker for spell checker</h4>
<a href = "https://pyspellchecker.readthedocs.io/en/latest/">Link to Pyspellchecker</a>

In [15]:
# import gingerit.gingerit
# from gingerit.gingerit import GingerIt

import spacy
from spellchecker import SpellChecker
import Levenshtein
import jellyfish
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Initialize the SpellChecker
spell = SpellChecker()

def cosine_similarity(str1, str2):
    vectorizer = TfidfVectorizer().fit_transform([str1, str2])
    vectors = vectorizer.toarray()
    vec1, vec2 = vectors[0], vectors[1]
    cosine_sim = vec1 @ vec2 / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return cosine_sim

def check_spelling(text):
    # Process the text with spaCy to get tokens
    doc = nlp(text)

    misspelled = {}
    for token in doc:
        if not token.is_punct and not token.is_space:
            word = token.text
            # Check if the word is misspelled
            if word.lower() not in spell:
                # Get the most likely correction
                correction = spell.correction(word)
                distance = Levenshtein.distance(correction.lower(), word.lower(), weights = (2, 1, 1))
                #  Compute the Jaro-Winkler distance
                # jw_distance = jellyfish.jaro_winkler(word, correction)
                misspelled[word] = {'correction': correction, 'distance': distance}


    return misspelled

# Example usage
text = "This is an exampel of a sentense with some erors. like youtube"
misspelled_words = check_spelling(text)
print("Misspelled words and corrections:", misspelled_words)

Misspelled words and corrections: {'exampel': {'correction': 'example', 'distance': 2}, 'sentense': {'correction': 'sentence', 'distance': 1}, 'erors': {'correction': 'errors', 'distance': 1}, 'youtube': {'correction': 'couture', 'distance': 2}}


<h4>Spello For Error Checking</h4>
<a href = "https://www.youtube.com/redirect?event=video_description&redir_token=QUFFLUhqbGVQNzl2aFJ5OVFhQUJMSWNxOUdUeHU0VGFtQXxBQ3Jtc0tscFZpQ1BvTnZtNUVqUWtKV1RUaDZZbTZUTjBNRVlBQnBrUFZyelNReUxZM1hIRXB5Y0FRcW10U1hkUFVmU0h3Zkc5dmxwazhuaUt0a01hd1NLSGpQMWthZUlyMG5SRTlCYko5allaMXlHQzdLU3EtVQ&q=https%3A%2F%2Fhaptik-website-images.haptik.ai%2Fspello_models%2Fen.pkl.zip&v=LEA3TDV98zE">Use this link to download pretrained model of spello</a>

In [16]:
from spello.model import SpellCorrectionModel
import pickle
sp = SpellCorrectionModel(language = "en")
sp.load("en.pkl")

D:\Leaning_NLP\myenv\Lib\site-packages\spello\model.py:301: UserWarning: This model was saved on spell<1.3.0. As such due to a bug in previous versions, none of customisations made to the config at the time of training were saved along with the model. It is recommended to load the model, apply all required customizations to config and save it again. E.g.

from spello.model import SpellCorrectionModel 
sp = SpellCorrectionModel(language='en')  
sp.load('/home/ubuntu/model.pkl')
sp.config.min_length_for_spellcorrection = 4 # default is 3
sp.config.max_length_for_spellcorrection = 12 # default is 15
sp.save(model_save_dir='/home/ubuntu/')

After this the model will load without any warnings

  warnings.warn("This model was saved on spell<1.3.0. As such due to a bug in previous versions, "


In [17]:
sp.spell_correct("My name is khan and I am not a terrorist")
sp.spell_correct("I want to google spanish amarican")

{'original_text': 'I want to google spanish amarican',
 'spell_corrected_text': 'I want to google spanish american',
 'correction_dict': {'amarican': 'american'}}

<h3>Using Spello with pretrained model gives best result.</h3>

In [18]:
from tqdm import tqdm

tqdm.pandas()

def spell_correction(text):
    p = sp.spell_correct(text)
    return p['spell_corrected_text']

df['overview'].progress_apply(spell_correction)
df['overview'].head(10)

100%|█████████████████████████████████████████████████████████████████████████| 2000/2000 [00:20<00:00, 96.44it/s]


0    imprisoned in the 1940s for the double murder ...
1    spanning the years 1945 to 1955 a chronicle of...
2    in the continuing saga of the corleone crime f...
3    the true story of how businessman oskar schind...
4    the defense and the prosecution have rested an...
5    raj is a rich carefree happygolucky second gen...
6    a young girl chihiro becomes trapped in a stra...
7    batman raises the stakes in his war on crime w...
8    all unemployed kitaeks family takes peculiar i...
9    a supernatural tale set on death row in a sout...
Name: overview, dtype: object

In [19]:
df['overview'][4]

'the defense and the prosecution have rested and the jury is filing into the jury room to decide if a young spanishamerican is guilty or innocent of murdering his father what begins as an open and shut case soon becomes a minidrama of each of the jurors prejudices and preconceptions about the trial the accused and each other'

<h3>Removing stop words</h3>

In [20]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
en = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
new_en = []
for word in en:
    if word != "not":
        new_en.append(word)

en = new_en

In [22]:
def remove_stopwords(text):
    new_text = []
    words = text.split(" ")
    for word in words:
        if word not in en:
            new_text.append(word)
    new_text = new_text[:]      
    return " ".join(new_text)       

In [23]:
remove_stopwords("my name is Khan and am not a terrorist")

'name Khan not terrorist'

In [24]:
df['overview'] = df['overview'].progress_apply(remove_stopwords)

100%|██████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 11587.48it/s]


In [25]:
df.head(10)

,original_language,original_title,overview,genre
0,en,The Shawshank Redemption,imprisoned 1940s double murder wife lover upst...,"[Drama, Crime]"
1,en,The Godfather,spanning years 1945 1955 chronicle fictional i...,"[Drama, Crime]"
2,en,The Godfather Part II,continuing saga corleone crime family young vi...,"[Drama, Crime]"
3,en,Schindler's List,true story businessman oskar schindler saved t...,"[Drama, History, War]"
4,en,12 Angry Men,defense prosecution rested jury filing jury ro...,[Drama]
5,hi,Dilwale Dulhania Le Jayenge (1995),raj rich carefree happygolucky second generati...,"[Comedy, Drama, Romance]"
6,ja,Spirited Away (2001),young girl chihiro becomes trapped strange new...,"[Animation, Family, Fantasy]"
7,en,The Dark Knight,batman raises stakes war crime help lt jim gor...,"[Drama, Action, Crime, Thriller]"
8,ko,Parasite (2019),unemployed kitaeks family takes peculiar inter...,"[Comedy, Thriller, Drama]"
9,en,The Green Mile,supernatural tale set death row southern priso...,"[Fantasy, Drama, Crime]"


In [26]:
df_temp = df

In [27]:
df = df_temp

<h3>Handeling Emojis</h3>

In [28]:
import emoji
def remove_emoji(text):
    return emoji.demojize(text)

print(remove_emoji("Hello 🔥"))

Hello :fire:


In [29]:
df['overview'] = df['overview'].progress_apply(remove_emoji)

100%|███████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 4403.30it/s]


<h3>Tokenization</h3>
<h4>We are using Spacy package</h4>

In [30]:
import spacy

nlp = spacy.load("en_core_web_sm")

def tokenizer(text):
    tokens = nlp(text)
    rtn = []
    for token in tokens:
        rtn.append(str(token))
    return rtn
    
print(tokenizer("My name is khan.. and i am not a terrorist!!"))

['My', 'name', 'is', 'khan', '..', 'and', 'i', 'am', 'not', 'a', 'terrorist', '!', '!']


In [31]:
df.sample(5)

,original_language,original_title,overview,genre
676,it,Non ci resta che piangere,two 20thcentury friends accidentally stumble y...,[Comedy]
927,en,In the Heat of the Night,africanamerican philadelphia police detective ...,"[Crime, Drama, Mystery, Thriller]"
1133,en,True Romance,clarence marries hooker alabama steals cocaine...,"[Action, Crime, Romance]"
805,en,Little Miss Sunshine,family loaded quirky colorful characters piles...,"[Comedy, Drama]"
1195,ja,ドラゴンボールZ たったひとりの最終決戦〜フリーザに挑んだZ戦士 孫悟空の父〜,bardock son gokus father lowranking saiyan sol...,"[Animation, Action, Science Fiction, Drama, Fa..."


In [32]:
df['overview'] = df['overview'].progress_apply(tokenizer)

100%|█████████████████████████████████████████████████████████████████████████| 2000/2000 [00:34<00:00, 58.43it/s]


In [33]:
df.head(5)

,original_language,original_title,overview,genre
0,en,The Shawshank Redemption,"[imprisoned, 1940s, double, murder, wife, love...","[Drama, Crime]"
1,en,The Godfather,"[spanning, years, 1945, 1955, chronicle, ficti...","[Drama, Crime]"
2,en,The Godfather Part II,"[continuing, saga, corleone, crime, family, yo...","[Drama, Crime]"
3,en,Schindler's List,"[true, story, businessman, oskar, schindler, s...","[Drama, History, War]"
4,en,12 Angry Men,"[defense, prosecution, rested, jury, filing, j...",[Drama]


<h3>Steming</h3>
<h4>We use ntlk PotterStemmer for doing this</h4>

In [34]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

def stemmer(words):
    rtn = []
    for word in words:
        rtn.append(ps.stem(word))

    return rtn

print(stemmer(["The", "world", "keeps", "on", "roating", "even", "if", "rejection", "awaits"]))

['the', 'world', 'keep', 'on', 'roat', 'even', 'if', 'reject', 'await']


In [35]:
df_temp['overview'] = df['overview'].progress_apply(stemmer)
# print(type(df['overview'][0][0]))

100%|███████████████████████████████████████████████████████████████████████| 2000/2000 [00:01<00:00, 1815.86it/s]


<h4>We can also do Lametization but would be little more time to complete but may lead to words in common  vocab for better understanding</h4>

In [38]:
df.to_csv("D:\\Leaning_NLP\\tmdb_data.csv")